In [61]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model, create_endpoint
import os
import tensorflow as tf

In [77]:
conn = ClipperConnection(DockerContainerManager())
conn.start_clipper()

18-06-21:21:17:39 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-06-21:21:17:44 INFO     [clipper_admin.py:126] Clipper is running


In [ ]:
conn.register_application(name='hello-world', input_type='doubles', default_output='-1.0', slo_micros=1000000)

In [ ]:
conn.delete_application(name='simple')

In [79]:
conn.register_application(name='simple', input_type='floats', default_output='-1.0', slo_micros=10000000)

18-06-21:21:18:26 ERROR    [clipper_admin.py:197] Received error status code: 400 and message: application 'simple' already exists


ClipperException: Received error status code: 400 and message: application 'simple' already exists

In [80]:
sess = tf.Session()
saver = tf.train.import_meta_graph('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model.meta')
saver.restore(sess, tf.train.latest_checkpoint('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/'))

INFO:tensorflow:Restoring parameters from /Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model


18-06-21:21:18:34 INFO     [tf_logging.py:82] Restoring parameters from /Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model


In [ ]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

In [ ]:
# dir = os.path.dirname(os.path.realpath(__file__))
dir = '.'

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the useful nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model_dir", type=str, default="", help="Model folder to export")
#     parser.add_argument("--output_node_names", type=str, default="", help="The name of the output nodes, comma separated.")
#     args = parser.parse_args()

#     freeze_graph(args.model_dir, args.output_node_names)

In [ ]:
freeze_graph('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/', 'predict_op')

In [4]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [ ]:
graph = load_graph('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/frozen_model.pb')

In [ ]:
for op in graph.get_operations():
    print(op.name)

In [ ]:
with tf.Session(graph=graph) as sess:
    y_hat = sess.run(y, feed_dict={x: [[ 0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.01176471
  ,0.07058824  ,0.07058824  ,0.07058824  ,0.49411765  ,0.53333336  ,0.68627453
  ,0.10196079  ,0.65098041  ,1.          ,0.96862745  ,0.49803922  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.11764706  ,0.14117648  ,0.36862746
  ,0.60392159  ,0.66666669  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.88235295  ,0.67450982  ,0.99215686  ,0.94901961  ,0.7647059
  ,0.25098041  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.19215687
  ,0.93333334  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.98431373  ,0.36470589  ,0.32156864
  ,0.32156864  ,0.21960784  ,0.15294118  ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.07058824  ,0.85882354  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.7764706   ,0.71372551  ,0.96862745  ,0.94509804
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.3137255   ,0.61176473
  ,0.41960785  ,0.99215686  ,0.99215686  ,0.80392158  ,0.04313726  ,0.
  ,0.16862746  ,0.60392159  ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.05490196  ,0.00392157  ,0.60392159  ,0.99215686  ,0.35294119  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.54509807  ,0.99215686  ,0.74509805  ,0.00784314
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.04313726  ,0.74509805  ,0.99215686
  ,0.27450982  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.13725491
  ,0.94509804  ,0.88235295  ,0.627451    ,0.42352942  ,0.00392157  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.31764707  ,0.94117647  ,0.99215686  ,0.99215686  ,0.46666667  ,0.09803922
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.17647059  ,0.72941178  ,0.99215686  ,0.99215686
  ,0.58823532  ,0.10588235  ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.0627451   ,0.36470589
  ,0.98823529  ,0.99215686  ,0.73333335  ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.97647059  ,0.99215686  ,0.97647059  ,0.25098041  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.18039216  ,0.50980395
  ,0.71764708  ,0.99215686  ,0.99215686  ,0.81176472  ,0.00784314  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.15294118  ,0.58039218  ,0.89803922
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.98039216  ,0.71372551  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.09411765  ,0.44705883  ,0.86666667  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.78823531  ,0.30588236  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.09019608  ,0.25882354  ,0.83529413  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.7764706   ,0.31764707  ,0.00784314  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.07058824  ,0.67058825  ,0.85882354  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.7647059   ,0.3137255   ,0.03529412  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.21568628  ,0.67450982  ,0.88627452  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.95686275  ,0.52156866  ,0.04313726  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.53333336  ,0.99215686  ,0.99215686  ,0.99215686  ,0.83137256
  ,0.52941179  ,0.51764709  ,0.0627451   ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.        ]]})
    print(y_hat[0])

In [81]:
#graph = load_graph('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/frozen_model.pb')

def predict(sess, inputs):
    return sess.run('predict_op:0', feed_dict={'input_X:0': inputs})


#     x = graph.get_tensor_by_name('prefix/input_X:0')
#     y = graph.get_tensor_by_name('prefix/predict_op:0')


In [ ]:
x = graph.get_tensor_by_name('prefix/input_X:0')
y = graph.get_tensor_by_name('prefix/predict_op:0')

In [82]:
tf.reset_default_graph()
sess = tf.Session()
saver = tf.train.import_meta_graph('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model.meta')
saver.restore(sess, tf.train.latest_checkpoint('/Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/'))

INFO:tensorflow:Restoring parameters from /Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model


18-06-21:21:18:51 INFO     [tf_logging.py:82] Restoring parameters from /Users/markmo/src/DeepLearning/dltemplate/src/tf_model/simple/data/model


In [84]:
predict(sess, [[ 0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.01176471
  ,0.07058824  ,0.07058824  ,0.07058824  ,0.49411765  ,0.53333336  ,0.68627453
  ,0.10196079  ,0.65098041  ,1.          ,0.96862745  ,0.49803922  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.11764706  ,0.14117648  ,0.36862746
  ,0.60392159  ,0.66666669  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.88235295  ,0.67450982  ,0.99215686  ,0.94901961  ,0.7647059
  ,0.25098041  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.19215687
  ,0.93333334  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.98431373  ,0.36470589  ,0.32156864
  ,0.32156864  ,0.21960784  ,0.15294118  ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.07058824  ,0.85882354  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.7764706   ,0.71372551  ,0.96862745  ,0.94509804
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.3137255   ,0.61176473
  ,0.41960785  ,0.99215686  ,0.99215686  ,0.80392158  ,0.04313726  ,0.
  ,0.16862746  ,0.60392159  ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.05490196  ,0.00392157  ,0.60392159  ,0.99215686  ,0.35294119  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.54509807  ,0.99215686  ,0.74509805  ,0.00784314
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.04313726  ,0.74509805  ,0.99215686
  ,0.27450982  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.13725491
  ,0.94509804  ,0.88235295  ,0.627451    ,0.42352942  ,0.00392157  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.31764707  ,0.94117647  ,0.99215686  ,0.99215686  ,0.46666667  ,0.09803922
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.17647059  ,0.72941178  ,0.99215686  ,0.99215686
  ,0.58823532  ,0.10588235  ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.0627451   ,0.36470589
  ,0.98823529  ,0.99215686  ,0.73333335  ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.97647059  ,0.99215686  ,0.97647059  ,0.25098041  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.18039216  ,0.50980395
  ,0.71764708  ,0.99215686  ,0.99215686  ,0.81176472  ,0.00784314  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.15294118  ,0.58039218  ,0.89803922
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.98039216  ,0.71372551  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.09411765  ,0.44705883  ,0.86666667  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.99215686  ,0.78823531  ,0.30588236  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.09019608  ,0.25882354  ,0.83529413  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.99215686  ,0.7764706   ,0.31764707  ,0.00784314  ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.07058824  ,0.67058825  ,0.85882354  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.7647059   ,0.3137255   ,0.03529412  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.21568628  ,0.67450982  ,0.88627452  ,0.99215686  ,0.99215686  ,0.99215686
  ,0.99215686  ,0.95686275  ,0.52156866  ,0.04313726  ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.53333336  ,0.99215686  ,0.99215686  ,0.99215686  ,0.83137256
  ,0.52941179  ,0.51764709  ,0.0627451   ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.          ,0.          ,0.
  ,0.          ,0.          ,0.          ,0.          ,0.        ]])

array([5])

In [66]:
create_endpoint(conn, 'simple2', 'floats', predict, sess)

18-06-19:00:25:55 INFO     [clipper_admin.py:201] Application simple2 was successfully registered
18-06-19:00:25:55 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmppzeidK
18-06-19:00:25:55 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-06-19:00:25:56 INFO     [tensorflow.py:191] TensorFlow model saved at: /tmp/clipper/tmppzeidK/tfmodel/model.ckpt 
18-06-19:00:25:56 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmppzeidK
18-06-19:00:25:56 INFO     [clipper_admin.py:456] {u'stream': u'Step 1/2 : FROM clipper/tf-container:develop'}
18-06-19:00:25:56 INFO     [clipper_admin.py:456] {u'stream': u'\n'}
18-06-19:00:25:56 INFO     [clipper_admin.py:456] {u'stream': u' ---> 5f65f2032eb5\n'}
18-06-19:00:25:56 INFO     [clipper_admin.py:456] {u'stream': u'Step 2/2 : COPY /tmp/clipper/tmppzeidK /model/'}
18-06-19:00:25:56 INFO     [clipper_admin.py:456] {u'stream': u'\n'}
18-06-19:00:25:56 INFO     [cli

In [85]:
deploy_tensorflow_model(conn, 'simple', 1, 'floats', predict, sess)

18-06-21:21:19:24 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpXlQYYA
18-06-21:21:19:24 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-06-21:21:19:24 INFO     [tensorflow.py:191] TensorFlow model saved at: /tmp/clipper/tmpXlQYYA/tfmodel/model.ckpt 
18-06-21:21:19:24 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpXlQYYA
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u'Step 1/2 : FROM clipper/tf-container:develop'}
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u'\n'}
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u' ---> 5f65f2032eb5\n'}
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u'Step 2/2 : COPY /tmp/clipper/tmpXlQYYA /model/'}
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u'\n'}
18-06-21:21:19:25 INFO     [clipper_admin.py:456] {u'stream': u' ---> 15f4a5c9ca63\n'}
18-06-21:21:19:25 INFO     [clipper_admin.

In [42]:
conn.get_all_apps()

[u'simple']

In [ ]:
def feature_sum(xs):
    return [str(sum(x)) for x in xs]

In [ ]:
python_deployer.deploy_python_closure(conn, name='predict', version=1, input_type='floats', func=predict)

In [86]:
conn.link_model_to_app(app_name='simple', model_name='simple')

18-06-21:21:19:57 INFO     [clipper_admin.py:263] Model simple is now linked to application simple


In [87]:
conn.get_query_addr()

'localhost:1337'

In [73]:
conn.get_linked_models(app_name="simple")

[]

In [74]:
conn.link_model_to_app(app_name="simple2", model_name="simple2")

18-06-21:08:41:40 ERROR    [clipper_admin.py:258] Received error status code: 400 and message: No app with name 'simple2' exists.


ClipperException: Received error status code: 400 and message: No app with name 'simple2' exists.

In [68]:
conn.get_clipper_logs()


['/Users/markmo/src/ML/clipper_test/clipper_logs/image_sha256:c8ecf7c719:container_b5f5ae98ca.log',
 '/Users/markmo/src/ML/clipper_test/clipper_logs/image_sha256:2fd23bd9c3:container_4c071fa5c4.log',
 '/Users/markmo/src/ML/clipper_test/clipper_logs/image_sha256:fa59587377:container_84078edd6e.log',
 '/Users/markmo/src/ML/clipper_test/clipper_logs/image_sha256:274991201d:container_ab1f71f899.log',
 '/Users/markmo/src/ML/clipper_test/clipper_logs/image_sha256:494c839f5b:container_d8e6eb3727.log']

## Cleanup

In [88]:
conn.stop_all()

18-06-21:21:22:14 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
